# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
import datetime

def imread(path):
    from PIL import Image
    return np.array(Image.open(path))

def imresize(img, size):
    from PIL import Image
    return np.array(Image.fromarray(np.uint8(img)).resize(size))

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 16

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
def generator(source_path, folder_list, batch_size):
    print('Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,1,2,4,6,7,8,10,12,13,14,16,18,19,20,22,24,25,26,28]
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t) // batch_size
        for batch in range(num_batches):
            batch_data = np.zeros((batch_size, 20, 120, 120, 3))
            batch_labels = np.zeros((batch_size, 5))
            for folder in range(batch_size):
                imgs = os.listdir('Project_data/'+source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0])
                for idx, item in enumerate(img_idx):
                    image = imread('Project_data/'+source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    image = imresize(image,(120, 120))
                                        
                    batch_data[folder,idx,:,:,0] = (image[:,:, 0])/255 
                    batch_data[folder,idx,:,:,1] = (image[:,:, 1])/255 
                    batch_data[folder,idx,:,:,2] = (image[:,:, 2])/255
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

        if (len(t) / batch_size) != 0:
            batch_data = np.zeros((batch_size, 20, 120, 120, 3))
            batch_labels = np.zeros((batch_size, 5))
            for folder in range(batch_size):
                imgs = os.listdir('Project_data/'+source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread('Project_data/'+source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imresize(image,(120, 120)).astype(np.float32)
                    batch_data[folder,idx,:,:,0] = (image[:,:, 0])/255
                    batch_data[folder,idx,:,:,1] = (image[:,:, 1])/255
                    batch_data[folder,idx,:,:,2] = (image[:,:, 2])/255
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [5]:
curr_dt_time = datetime.datetime.now()
train_path = 'train'
val_path = 'val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


## Models
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [6]:
# importing the libraries to be used

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers

#### Model #1 - Basic Conv3D

In [7]:
Input_Shape = (20, 120, 120, 3)

model = Sequential()

model.add(Conv3D(16, (3, 3, 3), padding='same', input_shape=(Input_Shape)))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten())
model.add(Dense(32, activation='elu'))
model.add(BatchNormalization())
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [8]:
opt = optimizers.Adam(learning_rate = 0.00001)
model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 20, 120, 120, 16)  1312      
_________________________________________________________________
activation (Activation)      (None, 20, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization (BatchNo (None, 20, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 10, 60, 60, 16)    0         
_________________________________________________________________
flatten (Flatten)            (None, 576000)            0         
_________________________________________________________________
dense (Dense)                (None, 32)                18432032  
_________________________________________________________________
batch_normalization_1 (Batch (None, 32)                1

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [13]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
model_name = 'GR_Model' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor = 'val_loss', verbose = 1, save_best_only = True, save_weights_only = False
                             , mode = 'auto', period=1)

LR = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 10, verbose = 1, mode = "auto", epsilon = 1e-06
                       , cooldown = 0, min_lr = 0)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [12]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [13]:
model.fit(train_generator, steps_per_epoch = steps_per_epoch, epochs = num_epochs, verbose = 1, callbacks = callbacks_list
          , validation_data = val_generator, validation_steps = validation_steps, class_weight = None, workers = 1
          , initial_epoch = 0)

Source path =  train ; batch size = 16
Epoch 1/20
42/42 [==============================] - 131s 3s/step - loss: 1.6564 - categorical_accuracy: 0.4120 - val_loss: 2.1960 - val_categorical_accuracy: 0.1875

Epoch 00001: val_loss improved from inf to 2.19596, saving model to GR_Model_2021-11-1408_27_02.838782/model-00001-1.40520-0.49107-2.19596-0.18750.h5
Epoch 2/20
42/42 [==============================] - 49s 1s/step - loss: 0.7214 - categorical_accuracy: 0.7452 - val_loss: 2.2660 - val_categorical_accuracy: 0.2321

Epoch 00002: val_loss did not improve from 2.19596
Epoch 3/20
42/42 [==============================] - 47s 1s/step - loss: 0.4198 - categorical_accuracy: 0.9074 - val_loss: 2.2946 - val_categorical_accuracy: 0.2946

Epoch 00003: val_loss did not improve from 2.19596
Epoch 4/20
42/42 [==============================] - 47s 1s/step - loss: 0.2587 - categorical_accuracy: 0.9562 - val_loss: 2.0656 - val_categorical_accuracy: 0.4018

Epoch 00004: val_loss improved from 2.19596 to 2

`Observation`: <b>As expected, base model is not performing well and is overfitting. Highest validation accuracy is 69.64%.</b>

#### Model #2 - Conv3D with Multiple Layers

In [29]:
model = Sequential()

model.add(Conv3D(16, (3, 3, 3), padding = 'same', input_shape = (Input_Shape)))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size = (2, 2, 2)))

model.add(Conv3D(32, (3, 3, 3), padding = 'same'))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size = (2, 2, 2)))

model.add(Conv3D(64, (3, 3, 3), padding = 'same'))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size = (2, 2, 2)))

model.add(Flatten())
model.add(Dense(128, activation = 'elu'))
model.add(BatchNormalization())
model.add(Dense(5, activation = 'softmax'))

In [30]:
opt = optimizers.Adam(learning_rate = 0.00001)
model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
print (model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_10 (Conv3D)           (None, 20, 120, 120, 16)  1312      
_________________________________________________________________
activation_10 (Activation)   (None, 20, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 20, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d_10 (MaxPooling (None, 10, 60, 60, 16)    0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 10, 60, 60, 32)    13856     
_________________________________________________________________
activation_11 (Activation)   (None, 10, 60, 60, 32)    0         
_________________________________________________________________
batch_normalization_15 (Batc (None, 10, 60, 60, 32)   

In [31]:
model_name = 'GR_Model' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor = 'val_loss', verbose = 1, save_best_only = True, save_weights_only = False
                             , mode = 'auto', period=1)

LR = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 4, verbose = 1, mode = "auto", epsilon = 1e-08
                       , cooldown = 0, min_lr = 0)
callbacks_list = [checkpoint, LR]

In [32]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [33]:
model.fit(train_generator, steps_per_epoch = steps_per_epoch, epochs = num_epochs, verbose = 1, callbacks = callbacks_list
          , validation_data = val_generator, validation_steps = validation_steps, class_weight = None, workers = 1
          , initial_epoch = 0)

Epoch 1/20
42/42 [==============================] - 49s 1s/step - loss: 1.9078 - categorical_accuracy: 0.2906 - val_loss: 1.9379 - val_categorical_accuracy: 0.2679

Epoch 00001: val_loss improved from inf to 1.93793, saving model to GR_Model_2021-11-1408_27_02.838782/model-00001-1.56632-0.41518-1.93793-0.26786.h5
Epoch 2/20
42/42 [==============================] - 47s 1s/step - loss: 0.8419 - categorical_accuracy: 0.6882 - val_loss: 2.8201 - val_categorical_accuracy: 0.1875

Epoch 00002: val_loss did not improve from 1.93793
Epoch 3/20
42/42 [==============================] - 47s 1s/step - loss: 0.6441 - categorical_accuracy: 0.7776 - val_loss: 3.9748 - val_categorical_accuracy: 0.2857

Epoch 00003: val_loss did not improve from 1.93793
Epoch 4/20
42/42 [==============================] - 47s 1s/step - loss: 0.4418 - categorical_accuracy: 0.8700 - val_loss: 4.1517 - val_categorical_accuracy: 0.2768

Epoch 00004: val_loss did not improve from 1.93793
Epoch 5/20
42/42 [===================

`Observation`: <b>Model is still overfitting. However, model performance has improved significantly. Highest validation accuracy is 79.46%. Average </b>

#### Model #3 - 3D Conv with Multiple Layers and Dropout

In [8]:
Input_Shape = (20, 120, 120, 3)

model = Sequential()

model.add(Conv3D(16, (3, 3, 3), padding = 'same', input_shape = (Input_Shape)))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size = (2, 2, 2)))

model.add(Conv3D(32, (3, 3, 3), padding = 'same'))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size = (2, 2, 2)))

model.add(Conv3D(64, (3, 3, 3), padding = 'same'))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size = (2, 2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64, activation = 'elu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(5, activation = 'softmax'))

In [9]:
opt = optimizers.Adam(learning_rate = 0.00001)
model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
print (model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 20, 120, 120, 16)  1312      
_________________________________________________________________
activation (Activation)      (None, 20, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization (BatchNo (None, 20, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 10, 60, 60, 16)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 10, 60, 60, 32)    13856     
_________________________________________________________________
activation_1 (Activation)    (None, 10, 60, 60, 32)    0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 60, 60, 32)   

In [10]:
model_name = 'GR_Model' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor = 'val_loss', verbose = 1, save_best_only = True, save_weights_only = False
                             , mode = 'auto', period=1)

LR = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 4, verbose = 1, mode = "auto", epsilon = 1e-08
                       , cooldown = 0, min_lr = 0)
callbacks_list = [checkpoint, LR]

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [14]:
model.fit(train_generator, steps_per_epoch = steps_per_epoch, epochs = num_epochs, verbose = 1, callbacks = callbacks_list
          , validation_data = val_generator, validation_steps = validation_steps, class_weight = None, workers = 1
          , initial_epoch = 0)

Source path =  train ; batch size = 16
Epoch 1/20
42/42 [==============================] - 130s 3s/step - loss: 2.2707 - categorical_accuracy: 0.2441 - val_loss: 1.6359 - val_categorical_accuracy: 0.2321

Epoch 00001: val_loss improved from inf to 1.63586, saving model to GR_Model_2021-11-1417_26_26.760695/model-00001-2.04268-0.29167-1.63586-0.23214.h5
Epoch 2/20
42/42 [==============================] - 48s 1s/step - loss: 1.4801 - categorical_accuracy: 0.4418 - val_loss: 2.4935 - val_categorical_accuracy: 0.2232

Epoch 00002: val_loss did not improve from 1.63586
Epoch 3/20
42/42 [==============================] - 47s 1s/step - loss: 1.2179 - categorical_accuracy: 0.5103 - val_loss: 3.2763 - val_categorical_accuracy: 0.2321

Epoch 00003: val_loss did not improve from 1.63586
Epoch 4/20
42/42 [==============================] - 47s 1s/step - loss: 1.0602 - categorical_accuracy: 0.5975 - val_loss: 4.0575 - val_categorical_accuracy: 0.2054

Epoch 00004: val_loss did not improve from 1.635

`Observation`: <b>With the addition of Dropout, overfitting has been addressed. Best validation accuracy for the model is 66.96%</b>

#### Model #4 - 3D Conv with Less Layers as Model #3

In [15]:
model = Sequential()

model.add(Conv3D(16, (2, 2, 2), padding='same', input_shape=(Input_Shape)))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(32, (2, 2, 2), padding='same'))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(32, activation='elu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(5, activation='softmax'))

In [16]:
opt = optimizers.Adam(learning_rate = 0.00001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_3 (Conv3D)            (None, 20, 120, 120, 16)  400       
_________________________________________________________________
activation_3 (Activation)    (None, 20, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 20, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 10, 60, 60, 16)    0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 10, 60, 60, 32)    4128      
_________________________________________________________________
activation_4 (Activation)    (None, 10, 60, 60, 32)    0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 10, 60, 60, 32)   

In [17]:
model_name = 'GR_Model' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor = 'val_loss', verbose = 1, save_best_only = True, save_weights_only = False
                             , mode = 'auto', period=1)

LR = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 4, verbose = 1, mode = "auto", epsilon = 1e-08
                       , cooldown = 0, min_lr = 0)
callbacks_list = [checkpoint, LR]

In [18]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [19]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
42/42 [==============================] - 49s 1s/step - loss: 2.0431 - categorical_accuracy: 0.3266 - val_loss: 1.7756 - val_categorical_accuracy: 0.2679

Epoch 00001: val_loss improved from inf to 1.77559, saving model to GR_Model_2021-11-1417_26_26.760695/model-00001-1.77104-0.39137-1.77559-0.26786.h5
Epoch 2/20
42/42 [==============================] - 46s 1s/step - loss: 1.0595 - categorical_accuracy: 0.5923 - val_loss: 2.7188 - val_categorical_accuracy: 0.2679

Epoch 00002: val_loss did not improve from 1.77559
Epoch 3/20
42/42 [==============================] - 47s 1s/step - loss: 0.9012 - categorical_accuracy: 0.6598 - val_loss: 3.3108 - val_categorical_accuracy: 0.3036

Epoch 00003: val_loss did not improve from 1.77559
Epoch 4/20
42/42 [==============================] - 47s 1s/step - loss: 0.7109 - categorical_accuracy: 0.7094 - val_loss: 3.6509 - val_categorical_accuracy: 0.2768

Epoch 00004: val_loss did not improve from 1.77559
Epoch 5/20
42/42 [===================

`Observation`: <b>There is not much changes as compared to model #3. However, the model overfits because we have removed the layer with Dropout. Best validation accuracy is 73.21%.</b>

#### Model #5 - Conv2D + RNN

In [52]:
Input_Shape = (20, 120, 120, 3)

model = Sequential()

model.add(TimeDistributed(Conv2D(16, (3, 3), padding='same', activation='relu'), input_shape=Input_Shape))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(Dropout(0.25))

model.add(TimeDistributed(Flatten()))

model.add(GRU(128))

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(5, activation='softmax'))

In [53]:
opt = optimizers.Adam(learning_rate = 0.00001)
model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
print (model.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_26 (TimeDis (None, 20, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_27 (TimeDis (None, 20, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_28 (TimeDis (None, 20, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_29 (TimeDis (None, 20, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_30 (TimeDis (None, 20, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_31 (TimeDis (None, 20, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_32 (TimeDis (None, 20, 30, 30, 64)   

In [54]:
model_name = 'GR_Model' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor = 'val_loss', verbose = 1, save_best_only = True, save_weights_only = False
                             , mode = 'auto', period=1)

LR = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 4, verbose = 1, mode = "auto", epsilon = 1e-08
                       , cooldown = 0, min_lr = 0)
callbacks_list = [checkpoint, LR]

In [55]:
model.fit(train_generator, steps_per_epoch = steps_per_epoch, epochs = num_epochs, verbose = 1, callbacks = callbacks_list
          , validation_data = val_generator, validation_steps = validation_steps, class_weight = None, workers = 1
          , initial_epoch = 0)

Epoch 1/20
42/42 [==============================] - 60s 1s/step - loss: 1.8543 - categorical_accuracy: 0.1954 - val_loss: 1.6166 - val_categorical_accuracy: 0.1518

Epoch 00001: val_loss improved from inf to 1.61658, saving model to GR_Model_2021-11-1408_27_02.838782/model-00001-1.76923-0.23065-1.61658-0.15179.h5
Epoch 2/20
42/42 [==============================] - 45s 1s/step - loss: 1.6561 - categorical_accuracy: 0.2605 - val_loss: 1.6144 - val_categorical_accuracy: 0.2143

Epoch 00002: val_loss improved from 1.61658 to 1.61441, saving model to GR_Model_2021-11-1408_27_02.838782/model-00002-1.62262-0.27976-1.61441-0.21429.h5
Epoch 3/20
42/42 [==============================] - 47s 1s/step - loss: 1.4978 - categorical_accuracy: 0.3460 - val_loss: 1.6520 - val_categorical_accuracy: 0.1964

Epoch 00003: val_loss did not improve from 1.61441
Epoch 4/20
42/42 [==============================] - 46s 1s/step - loss: 1.4231 - categorical_accuracy: 0.3961 - val_loss: 1.6721 - val_categorical_acc

`Observation`: <b>Conv2D + RNN model is not performing well but has no overfitting. Best validation accuracy is 57.14%.</b>